In [47]:
from sdm.utils import set_project_wd
set_project_wd()

Current Working Directory: /Users/matthewwhittle/Data Science/shefflied-bats


In [48]:
from pathlib import Path
ev_folder = Path("data/evs/")
# list the tifs
ev_tifs = list(ev_folder.glob("*.tif"))
ev_tifs

[PosixPath('data/evs/climate_stats.tif'),
 PosixPath('data/evs/os-feature-cover-100m.tif'),
 PosixPath('data/evs/os-distance-to-feature.tif'),
 PosixPath('data/evs/terrain-stats.tif'),
 PosixPath('data/evs/bioclim.tif'),
 PosixPath('data/evs/temp_average.tif'),
 PosixPath('data/evs/precipitation.tif'),
 PosixPath('data/evs/ceh-land-cover-100m.tif'),
 PosixPath('data/evs/dtm-100m.tif')]

In [49]:
import rioxarray as rxr
import xarray as xr

# read the tifs into a list of xarray objects
evs = [rxr.open_rasterio(ev_tif, band_as_variable=True) for ev_tif in ev_tifs]
# Name each EV using the filename
for ev, ev_tif in zip(evs, ev_tifs):
    ev.attrs["file_name"] = ev_tif.stem

In [50]:
evs[0].attrs

{'AREA_OR_POINT': 'Area', 'file_name': 'climate_stats'}

In [57]:
import numpy as np
np.isinf(evs[2]["band_1"]).any()

<xarray.DataArray 'band_1' ()>
array(False)
Coordinates:
    spatial_ref  int64 0

In [32]:
def assign_band_names(ds):
    if "long_name" not in ds["band_1"].attrs:
        return ds
    band_names = {var: ds[var].attrs["long_name"] for var in ds.data_vars}
    renamed_ds = ds.rename(band_names)
    return renamed_ds

evs = [assign_band_names(ev) for ev in evs]

In [34]:
# Prefix band names with the file_name for each dataset
evs = [ev.rename({var: f"{ev.attrs['file_name']}_{var}" for var in ev.data_vars}) for ev in evs]

In [35]:
ev_stack = xr.merge(evs)
ev_stack

<xarray.Dataset>
Dimensions:                                         (x: 792, y: 563)
Coordinates:
  * x                                               (x) float64 4.039e+05 ......
  * y                                               (y) float64 4.268e+05 ......
    spatial_ref                                     int64 0
Data variables: (12/91)
    climate_stats_temp_ann_var                      (y, x) float64 ...
    climate_stats_temp_ann_avg                      (y, x) float64 ...
    climate_stats_temp_mat_avg                      (y, x) float64 ...
    climate_stats_prec_ann_var                      (y, x) float64 ...
    climate_stats_prec_ann_avg                      (y, x) float64 ...
    climate_stats_wind_ann_var                      (y, x) float64 ...
    ...                                              ...
    ceh-land-cover-100m_Littoral rock               (y, x) float64 ...
    ceh-land-cover-100m_Littoral sediment           (y, x) float64 ...
    ceh-land-cover-100m_Saltmarsh                   (y, x) float64 ...
    ceh-land-cover-100m_Urban                       (y, x) float64 ...
    ceh-land-cover-100m_Suburban                    (y, x) float64 ...
    dtm-100m_LIDAR_Composite_2m_DTM_2022_Elevation  (y, x) float32 ...
Attributes:
    AREA_OR_POINT:  Area
    file_name:      climate_stats

In [36]:
ev_stack.to_dataframe()

spatial_ref  climate_stats_temp_ann_var  \
x             y                                                        
403914.879459 426796.145539            0                         NaN   
              426696.051974            0                         NaN   
              426595.958408            0                         NaN   
              426495.864843            0                         NaN   
              426395.771277            0                         NaN   
...                                  ...                         ...   
483069.223985 370943.936046            0                    4.532321   
              370843.842481            0                    4.532317   
              370743.748915            0                    4.532313   
              370643.655350            0                    4.532308   
              370543.561784            0                    4.532304   

                             climate_stats_temp_ann_avg  \
x             y                                           
403914.879459 426796.145539                         NaN   
              426696.051974                         NaN   
              426595.958408                         NaN   
              426495.864843                         NaN   
              426395.771277                         NaN   
...                                                 ...   
483069.223985 370943.936046                    9.621739   
              370843.842481                    9.621713   
              370743.748915                    9.621687   
              370643.655350                    9.621661   
              370543.561784                    9.621635   

                             climate_stats_temp_mat_avg  \
x             y                                           
403914.879459 426796.145539                         NaN   
              426696.051974                         NaN   
              426595.958408                         NaN   
              426495.864843                         NaN   
              426395.771277                         NaN   
...                                                 ...   
483069.223985 370943.936046                   11.100001   
              370843.842481                   11.100001   
              370743.748915                   11.100001   
              370643.655350                   11.100001   
              370543.561784                   11.100001   

                             climate_stats_prec_ann_var  \
x             y                                           
403914.879459 426796.145539                         NaN   
              426696.051974                         NaN   
              426595.958408                         NaN   
              426495.864843                         NaN   
              426395.771277                         NaN   
...                                                 ...   
483069.223985 370943.936046                    4.913134   
              370843.842481                    4.913134   
              370743.748915                    4.913134   
              370643.655350                    4.913134   
              370543.561784                    4.913134   

                             climate_stats_prec_ann_avg  \
x             y                                           
403914.879459 426796.145539                         NaN   
              426696.051974                         NaN   
              426595.958408                         NaN   
              426495.864843                         NaN   
              426395.771277                         NaN   
...                                                 ...   
483069.223985 370943.936046                   45.833333   
              370843.842481                   45.833333   
              370743.748915                   45.833333   
              370643.655350                   45.833333   
              370543.561784                   45.833333   

                    

In [42]:
ev_stack.to_dataframe().corr()

,spatial_ref,climate_stats_temp_ann_var,climate_stats_temp_ann_avg,climate_stats_temp_mat_avg,climate_stats_prec_ann_var,climate_stats_prec_ann_avg,climate_stats_wind_ann_var,climate_stats_wind_ann_avg,os-feature-cover-100m_major_roads,os-feature-cover-100m_minor_roads,...,ceh-land-cover-100m_Saltwater,ceh-land-cover-100m_Freshwater,ceh-land-cover-100m_Supralittoral rock,ceh-land-cover-100m_Supralittoral sediment,ceh-land-cover-100m_Littoral rock,ceh-land-cover-100m_Littoral sediment,ceh-land-cover-100m_Saltmarsh,ceh-land-cover-100m_Urban,ceh-land-cover-100m_Suburban,dtm-100m_LIDAR_Composite_2m_DTM_2022_Elevation
spatial_ref,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
climate_stats_temp_ann_var,NaN,1.000000,0.834990,0.863242,-0.873251,-0.867353,NaN,NaN,0.004116,0.095285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.835299
climate_stats_temp_ann_avg,NaN,0.834990,1.000000,0.996161,-0.914489,-0.949602,NaN,NaN,0.014932,0.132289,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.974567
climate_stats_temp_mat_avg,NaN,0.863242,0.996161,1.000000,-0.912649,-0.945095,NaN,NaN,0.013124,0.137273,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.969565
climate_stats_prec_ann_var,NaN,-0.873251,-0.914489,-0.912649,1.000000,0.991492,NaN,NaN,-0.032653,-0.047363,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.959697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ceh-land-cover-100m_Littoral sediment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ceh-land-cover-100m_Saltmarsh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ceh-land-cover-100m_Urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ceh-land-cover-100m_Suburban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
